In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

raw_air_quality_df = spark.table("weather.bronze.bronze_air_quality")
raw_air_quality_df.display()

In [0]:
json_string = raw_air_quality_df.select("raw_air_quality_data").first()[0]
air_quality_schema = schema_of_json(json_string)

In [0]:
prased_air_quality_df = (
    raw_air_quality_df
    .withColumn("air_quality_data", from_json(col("raw_air_quality_data"), air_quality_schema))
)

prased_air_quality_df.display()

In [0]:
exploded_df = (
    prased_air_quality_df.select(
        "city",
        "ingestion_ts",
        posexplode(col("air_quality_data.hourly.time")).alias("pos", "event_time"),
        col("air_quality_data.hourly.pm2_5")[col("pos")].alias("pm2_5"),
        col("air_quality_data.hourly.pm10")[col("pos")].alias("pm10"),
        "source"
    )
)

exploded_df.display()

In [0]:
silver_air_quality = (
    exploded_df.dropDuplicates(["city", "event_time"])
)

In [0]:
(
    silver_air_quality.write
    .format("delta")
    .mode("overwrite")
    .save("/Volumes/weather/silver/data_files/air_quality/")
)